In [1]:
%load_ext autoreload
%autoreload 2
from configs import project_config
import matplotlib.pyplot as plt
from src.classes.ClassFractionalSin import FractionalSin
from tqdm import tqdm
import numpy as np
import math
from scipy.special import eval_laguerre
from configs.RecursiveVPSDE.Markovian_fSin.recursive_Markovian_fSinWithPosition_T256_H05_tl_5data import get_config

# From A ridge estimator of the drift from discrete repeated observations of the solutions of a stochastic differential equation (Denis et al 2021)

In [2]:
config = get_config()

In [3]:
def rmse_ignore_nans(y_true, y_pred):
    assert (y_true.shape == y_pred.shape and len(y_pred.shape) == 1)
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)  # Ignore NaNs in both arrays
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

In [ ]:
if "fQuadSin" in config.data_path:
    minx = -1.7
elif "fBiPot" in config.data_path:
    minx = -2
elif "fSin" in config.data_path:
    minx = -3
maxx = -minx
vispaths = np.linspace(minx, maxx, 256)
num_paths = 10152
if "QuadSin" in config.data_path:
    is_true_drift = 2.*config.quad_coeff * vispaths.flatten() + config.sin_coeff * config.sin_space_scale*np.sin(config.sin_space_scale*vispaths.flatten())
elif "fSin" in config.data_path:
    is_true_drift = config.mean_rev*np.sin(vispaths.flatten())
elif "BiPot" in config.data_path:
    is_true_drift = -(4.*config.quartic_coeff * np.power(vispaths, 3) + 2.*config.quad_coeff * vispaths + config.const).flatten()
bw =

In [5]:
save_path = (
        project_config.ROOT_DIR + f"experiments/results/IIDNadaraya_fQuadSin_DriftEvalExp_{round(bw, 4)}bw_{num_paths}NPaths").replace(
    ".", "")

In [13]:
is_drift = np.load(save_path + "_IIDNadaraya_isdriftTrue.npy")

In [ ]:
fig, ax = plt.subplots(figsize=(14,9))
plt.scatter(vispaths, is_true_drift,label="True Drift", color="red")
plt.scatter(vispaths, is_drift, label="Estimated Drift", color="blue")
plt.legend(fontsize=18)
rmse = (rmse_ignore_nans(is_true_drift, is_drift))
plt.title(f"RMSE {round(rmse, 4)} for Nadaraya Estimator", fontsize=40)
ax.tick_params(labelsize=38)
plt.ylabel("Drift Value", fontsize=48)
plt.xlabel(r"State $X$", fontsize=48)
plt.legend(fontsize=24)
plt.savefig(f"/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModelPresentationImages/fQuadSin_Nadaraya.png",  bbox_inches='tight')
plt.show()
plt.close()